In [1]:
import sys

sys.path.append("../")

from loaders.LHCO import LhcoRnDLoader
from analysis.scalar import HLFAnalysis
from models.nn import PaeBuilder
from utils import load_json, dump_json

import tensorflow as tf
import numpy as np

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [7]:
import glob

configs_folder = '../configs/analysis'
config_files = glob.glob(configs_folder+'/*.json')
config_files = ['../configs/configs/c3d4.json']

In [8]:
config_files[0].replace('.', '/').split('/')[-2]

'c3d4'

In [9]:
import os

plots_dir = './figures/'
if not os.path.exists(plots_dir):
    os.mkdir(plots_dir)
results_history = {
 'id': [],
 'config': [],
 'js_div_train': [],
 'js_div_test': [],
 'sig_eff': [],
 'bkg_rej': [],
 'auc': []}

In [13]:
 for file in config_files:
    analysis_cfg = load_json(file)
    analysis_cfg

    analysis_cfg["NF:n_dims"] = analysis_cfg["AE:encoding_dim"]
    config = analysis_cfg.copy()

    loader_json = analysis_cfg.pop('ANA:loader')
    dataset_json = analysis_cfg.pop('ANA:dataset')
    density_estimator = analysis_cfg.pop('ANA:estimator')
    prc = analysis_cfg.pop('ANA:percentile')

    loader = LhcoRnDLoader.from_json(loader_json)
    dataset_cfg = load_json(dataset_json)
    dataset = loader.make_full_dataset(**dataset_cfg)

    builder = PaeBuilder()

    pae, ae_train, nf_train = builder.from_json(analysis_cfg)
    task = HLFAnalysis(pae, dataset=dataset)
    task.reweighting(estimator=density_estimator, fit_key='mjj_train')
    if 'cond' in config['nf_model']:
        task.make_cond_inputs(['mjj_train', 'mjj_test', 'mjj_valid'])
    task.train(ae_train,nf_train)
    result = task.evaluate(prc = prc)
    for key in result.keys():
        results_history[key].append(result[key])
    
    id = config_files[0].replace('.', '/').split('/')[-2]
    results_history['id'].append(id)
    results_history['config'].append(config)
    task.plot_training(plots_dir+id+'_train.png')
    task.plot_latent_space(plots_dir+id+'_latent.png')



[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
800/800 [==============================] - 6s 7ms/step - loss: 696.7478 - val_loss: 1006.4934


ValueError: in user code:

    /home/ioan/.local/share/virtualenvs/tf-gpu-qnR-nPlh/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    ../models/flows.py:96 call  *
        return self.distribution.log_prob(x)
    /home/ioan/.local/share/virtualenvs/tf-gpu-qnR-nPlh/lib/python3.8/site-packages/tensorflow_probability/python/distributions/distribution.py:1030 log_prob  **
        return self._call_log_prob(value, name, **kwargs)
    /home/ioan/.local/share/virtualenvs/tf-gpu-qnR-nPlh/lib/python3.8/site-packages/tensorflow_probability/python/distributions/distribution.py:1012 _call_log_prob
        return self._log_prob(value, **kwargs)
    /home/ioan/.local/share/virtualenvs/tf-gpu-qnR-nPlh/lib/python3.8/site-packages/tensorflow_probability/python/distributions/transformed_distribution.py:364 _log_prob
        base_log_prob = self.distribution.log_prob(x, **distribution_kwargs)
    /home/ioan/.local/

In [ ]:
results_history